In [1]:
import numpy as np
import pandas as pd
import re

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [3]:
test.sample(10)

,id,comment_text
107343,b31bbf1cff4149fb,== December 2008 == \n Please stop. If you co...
54446,5a7fb894b5d99a07,""" \n\n == Response to your ideas on HappyApple..."
95214,9edd56efc3591528,==You're way to bias and not fair play== \n\n ...
82173,891e34995c32f120,==League Cups in other countries== \n Other co...
144093,f0cf53102aeb257c,""" \n *50/50 for me Certainly most of the capit..."
56335,5db576edd17570cd,""":Restored, since I wasn't done replying. Besi..."
69120,730e7dad21bba886,"""Hi, you recently commented on the VfD for Bri..."
109065,b5f2196205f275ee,":After looking at it further, I realized that ..."
124207,cf6d664ef00f31bf,"Damn, this really sucks... I hope this fightin..."
8151,0dab71977163c24e,== Drugs of the Future == \n\n is a secondary...


### The most common word

In [4]:
def get_words(text):
    result = re.findall(r"[a-z]+'?[a-z]+", text.lower())

    return result

In [5]:
train_text = train['comment_text'].apply(get_words)

test_text = test['comment_text'].apply(get_words)

all_text = pd.concat([train_text, test_text])

In [6]:
total = list()
for line in all_text:
    total.extend(line)

In [7]:
count = Counter(total)
count.most_common(3)

[('the', 919035), ('to', 539236), ('of', 410839)]

Какое слово встречается чаще всего в объединенном train и test датасете? - **'the'** 

### Увеличение параметра C в Logistic regression увеличивает или уменьшает степень регуляризации?

Увеличение параметра C в Logistic regression увеличивает или уменьшает степень регуляризации?

**C** – обратный коэффициент регуляризации (тот самый C в sklearn-реализации LogisticRegression)

Ответ: **Уменьшает**

### Crossvalidation

In [8]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [9]:
train_text = train_text.apply(lambda text: ' '.join(text))

test_text = test_text.apply(lambda text: ' '.join(text))

all_text = pd.concat([train_text, test_text])

In [10]:
word_vectorizer = TfidfVectorizer() # TfidfVectorizer или CountVectorizer

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [11]:
C = 3.2

classifier = LogisticRegression(C=C,random_state=7) 

scores= []

for class_name in class_names:
    
    train_target = train[class_name]

    cv_score = np.mean(cross_val_score(classifier, train_word_features, train_target, scoring='roc_auc'))

    print('CV score for class {} is {}'.format(class_name, cv_score))
    scores.append(cv_score)
    
print('Total score is {}'.format(np.mean(scores)))



CV score for class toxic is 0.9685448434591195
CV score for class severe_toxic is 0.9820677623440385
CV score for class obscene is 0.9832046607734969
CV score for class threat is 0.9828801193999221
CV score for class insult is 0.9737086129382343
CV score for class identity_hate is 0.9720986621263531
Total score is 0.9770841101735274


```
C = 3.2
word_vectorizer = TfidfVectorizer()
Total score is 0.9792841065937047
```

### Create submission for Kaggle

In [12]:
submission = pd.DataFrame.from_dict({'id': test['id']})

In [13]:
for class_name in class_names:  
    
    train_target = train[class_name]
    
    classifier = LogisticRegression(C=C,random_state=7)
      
    classifier.fit(train_word_features, train_target)
    
    submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

In [14]:
# submission.to_csv('submission.csv', index=False)

In [15]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999519,0.183758,0.998195,0.072261,0.962329,0.301808
1,0000247867823ef7,0.002827,0.000982,0.001422,0.000164,0.003538,0.002218
2,00013b17ad220c46,0.030333,0.004320,0.012821,0.001170,0.015787,0.003999
3,00017563c3f7919a,0.001273,0.001621,0.001600,0.000495,0.002601,0.000351
4,00017695ad8997eb,0.016390,0.002586,0.005237,0.001330,0.006241,0.001918
